# Getting Started With the HDAWG

## Device Setup and Data Server

Let's start by creating a `DeviceSetup` with a single HDAWG instrument. To learn more about what a `DeviceSetup` is and how to use it, have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/index.html).

First, we import `laboneq.simple`, which contains the `DeviceSetup`.

In [ ]:
import numpy as np

from laboneq.contrib.example_helpers.plotting.plot_helpers import plot_simulation
from laboneq.simple import *

Create the `DeviceSetup` and add the information about the data server it should connect to. Note that calling `add_dataserver` does not attempt the connection yet. This will be done only upon calling `Session.connect`.

In [ ]:
device_setup = DeviceSetup("ZI_HDAWG")
device_setup.add_dataserver(
    host="localhost",
    port="8004",
)

Create a `HDAWG` instrument, which was imported from `laboneq.simple`, and add it to the `DeviceSetup`. 

When creating the instrument instance, you need to specify the device ID under `address`; for example, `dev8123`.

If you do not have an active LabOne data server running to connect to the instrument, you also need to specify the `device_options` that are installed on your instrument. These options are used to ensure a correct experiment compilation for your system. The possible options you can set for an HDAWG instrument are:

* either `"HDAWG8"` if you have an 8-channel instrument or `"HDAWG4"` for the 4-channel version;
* `"CNT"` for the pulse counter option;
* `"MF"` for the multi-frequency option;
* `"ME"` for the memory extension option;
* `"PC"` for the real-time precompensation option;
* `"SKW"` for the output skew control option.

When passing these options to the instrument, add them in any order, separated by a forward slash (`"/"`). Below, we will use `device_options="HDAWG8/MF/CNT/PC"`. Have a look at [this page](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/00_device_setup/concepts/01_instrument_options.html) to learn more about the possible options of our instruments.

You can also pass additional input parameters to configure your instrument:
  
* the `interface` over which to connect to the instrument, either `"1GbE"` (default) or `"usb"`. Note that **to ensure the stability of the connection to the instrument, we recommend to use the ethernet interface instead of USB**;

* the `reference_clock_source` for the instrument, either as `"internal"` to use the instrument's own internal reference clock, or `"external"` (default) if you are using an external source like the PQSC instrument, for example.

In [ ]:
hdawg = HDAWG(
    uid="hdawg",
    interface="1GbE",
    address="dev8123",
    device_options="HDAWG8/MF/CNT/PC",
    reference_clock_source="internal",
)

device_setup.add_instruments(hdawg)

Next, we create connections to each of the 8 ports of the instrument and add these connections to the `DeviceSetup`. These connections are represented in LabOne Q as [logical signal lines](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/02_logical_signals/concepts/index.html) between the instruments and your device under test. Here, we assume the device under test is a six-qubit QPU and use the physical lines of these qubits as a naming convention for our signal lines. Note that the instances of the `LogicalSignal`s will be created automatically by the `add_connections` method of `DeviceSetup`, with the names that we have specified.

In [ ]:
# Create flux lines for 6 qubits (signal type is "rf")
for idx in range(4):
    device_setup.add_connections(
        "hdawg",
        create_connection(to_signal=f"q{idx}/flux", ports=f"SIGOUTS/{idx}", type="rf"),
    )

# Create a drive line for two qubits that use two ports of the HDAWG (signal type is "iq")
device_setup.add_connections(
    "hdawg",
    create_connection(
        to_signal="q4/drive", ports=["SIGOUTS/4", "SIGOUTS/5"], type="iq"
    ),
)
device_setup.add_connections(
    "hdawg",
    create_connection(
        to_signal="q5/drive", ports=["SIGOUTS/6", "SIGOUTS/7"], type="iq"
    ),
)

You can inspect the `LogicalSignal`s that have been created by calling `device_setup.logical_signal_by_uid(signal_name)`; for example:

In [ ]:
device_setup.logical_signal_by_uid("q4/drive")

Next, we configure the `Calibration` of the signal lines of the `DeviceSetup`. We will set a few common properties:

* `range` - the power range in volt of the output ports. Here we will set 5 V;
* `voltage_offset` - the DC voltage offset to be played on each channel. Here we set 0 V for all channels;
* `oscillator` - an instance of `Oscillator` where we specify the IF frequency that will modulate the pulses that are played back. Here we choose 200 MHz.

In [ ]:
config = Calibration()
for idx in range(4):
    config[f"q{idx}/flux"] = SignalCalibration(voltage_offset=0.0, range=5.0)

config["q5/drive"] = SignalCalibration(
    oscillator=Oscillator(frequency=200e6), voltage_offset=0.0, range=5.0
)
config["q4/drive"] = SignalCalibration(
    oscillator=Oscillator(frequency=200e6), voltage_offset=0.0, range=5.0
)

# Apply the configuration to the DeviceSetup
device_setup.set_calibration(config)

Great! We have our `DeviceSetup` for a single HDAWG instrument.

Before we can play a signal on the instrument, we first have to connect it to the LabOne data server via the LabOne Q [Session](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/01_session/concepts/index.html). Here, we connect in emulation mode by calling `Session.connect` with `do_emulation=True`. Set this flag to False in order to connect to a physical setup.

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)  # do_emulation=False when at a physical setup

## Generate a Simple Playback Experiment

Let's create a simple experiment that plays back a 200-ns gaussian pulse on all 8 drive channels. In addition, we will sweep the amplitude of the pulses.

To learn more about the `Experiment` object and how to write experiments in LabOne Q, have a look at the ["Experiment Definition"](https://docs.zhinst.com/labone_q_user_manual/core/functionality_and_concepts/05_experiment/concepts/index.html) and ["Writing and Experiment Workflow"](https://docs.zhinst.com/labone_q_user_manual/applications_library/tutorials/sources/writing_experiments.html#write-the-experiment-pulse-sequence) sections of the manual.

In [ ]:
@dsl.experiment(signals=[f"q{idx}_flux" for idx in range(4)] + ["q4_drive"])
def simple_experiment(count):
    with dsl.acquire_loop_rt(
        count=count,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with dsl.sweep(
            name="amplitude_sweep",
            parameter=SweepParameter("drive_pulse_amplitudes", np.linspace(0, 1, 7)),
        ) as amplitude:
            for idx in range(4):
                with dsl.section(name=f"play-drive-pulse_q{idx}"):
                    dsl.play(
                        f"q{idx}_flux",
                        pulse_library.gaussian(amplitude=1, length=200e-9),
                        amplitude=amplitude,
                    )
                    dsl.delay(f"q{idx}_flux", time=0.5e-6)

            with dsl.section(name="play-drive-pulse_q4"):
                dsl.play(
                    "q4_drive",
                    pulse_library.drag(amplitude=1, length=200e-9, beta=0.2),
                    amplitude=amplitude,
                )
                dsl.delay("q4_drive", time=0.5e-6)

    # Map the ExperimentSignals "q{idx}_flux", "q{idx}_drive" to the logical signal lines defined in the `DeviceSetup`
    for idx in range(4):
        dsl.map_signal(f"q{idx}_flux", f"q{idx}/flux")
    dsl.map_signal("q4_drive", "q4/drive")

Next, we instantiate the `Experiment` by running the function `simple_experiment`.

In [ ]:
experiment = simple_experiment(5)

Note that, the mapping between the `ExperimentSignal`s and the logical signal lines do not have to be done as part of `simple_experiment` but can also be done or modified on the `Experiment` instance returned by `simple_experiment` as follows:

```python
q0_ls = device_setup.logical_signal_groups["q0"].logical_signals
experiment.map_signal("q0_flux", q0_ls["flux"])
q4_ls = device_setup.logical_signal_groups["q4"].logical_signals
experiment.map_signal("q4_drive", q4_ls["drive"])
```

You can also map an `ExperimentSignal` directly to a `LogicalSignal` instead of its UID, as written above.

Compile the simple_experiment and inspect it using `plot_simulation`:

In [ ]:
compiled_experiment = session.compile(experiment)

In [ ]:
plot_simulation(compiled_experiment, start_time=0, length=5e-6)

You can inspect your pulse sequence in more detail by using the interactive pulse-sheet viewer. Calling the function `show_pulse_sheet` creates an HTML file that you can open in your browser. To show the pulse sheet in the kernel, set `interactive=True`.

In [ ]:
show_pulse_sheet("simple_pulse_sequence", compiled_experiment, interactive=True)

Finally, let's run the pulse sequence on the instrument.

In [ ]:
_ = session.run()

## Using Markers and Triggers

Let's create a simple experiment that plays back a marker and a trigger output in addition to the waveforms. Below, we create an experiment where:

* we use an `rf` signal line (`q0/flux`) to play a 200-ns Gaussian pulse on `SIGOUT/0` of the HDAWG, accompanied by a constant pulse of the same length on the marker output of `SIGOUT/0`. We enable the marker by using "enable";

* we use an `rf` signal line (`q1/flux`) to play a 200-ns Gaussian pulse on `SIGOUT/1` of the HDAWG, accompanied by a 1-$\mu$s output on the corresponding trigger port;

* we use an `iq` signal line (`q4/drive`) to play a 200-ns DRAG pulse on the HDAWG ports [`SIGOUT/4`, `SIGOUT/5`], accompanied by a 100-ns constant pulse on the marker of `SIGOUT/4` (`"marker1"`), and a 200-ns constant pulse on the marker of `SIGOUT/5` (`"marker2"`). We specify `"marker1"` using `"start"` and `"length"`, and `"marker2"` as a waveform;

* we use an `iq` signal line (`q5/drive`) to play a 200-ns DRAG pulse on the HDAWG ports [`SIGOUT/4`, `SIGOUT/5`], accompanied by a 1-$\mu$s output on each of the two corresponding trigger ports by setting the `"state"` to 3. Use 1 to produce an output only on the trigger port of `SIGOUT/4`, or 2 to produce an output only on the trigger port of `SIGOUT/5`.

To learn more details about configuring markers and trigger in LabOne Q, have a look at this page [ADD LINK].

In [ ]:
@dsl.experiment(signals=["q0_flux", "q1_flux", "q4_drive", "q5_drive"])
def experiment_trig_mark(count):
    with dsl.acquire_loop_rt(
        count=count,
        averaging_mode=AveragingMode.CYCLIC,
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        # Marker on rf channel
        with dsl.section(name="rf-signal_q0"):
            dsl.play(
                signal="q0_flux",
                pulse=pulse_library.gaussian(amplitude=1, length=200e-9),
                marker={"marker1": {"enable": True}},
            )
            dsl.delay("q0_flux", time=0.5e-6)

        # Trigger on rf channel
        with dsl.section(
            name="rf-signal_q1", length=1e-6, trigger={"q1_flux": {"state": 1}}
        ):
            dsl.play(
                signal="q1_flux",
                pulse=pulse_library.gaussian(amplitude=1, length=200e-9),
            )
            dsl.delay("q1_flux", time=0.5e-6)

        # Markers on iq channel
        with dsl.section(name="iq_signal_q4"):
            dsl.play(
                "q4_drive",
                pulse_library.drag(amplitude=1, length=200e-9, beta=0.2),
                marker={
                    "marker1": {"start": 100e-9, "length": 100e-9},
                    "marker2": {
                        "waveform": pulse_library.const(amplitude=0.5, length=200e-9)
                    },
                },
            )
            dsl.delay("q4_drive", time=0.5e-6)

        # Trigger on iq channel
        with dsl.section(
            name="iq_signal_q5", length=1e-6, trigger={"q5_drive": {"state": 3}}
        ):
            dsl.play(
                "q5_drive",
                pulse_library.drag(amplitude=1, length=400e-9, beta=0.2),
            )

    # Map the ExperimentSignals "q0_flux", "q1_flux", "q4_drive", "q5_drive" to the logical signal line names defined in the `DeviceSetup`
    dsl.map_signal("q0_flux", "q0/flux")
    dsl.map_signal("q1_flux", "q1/flux")
    dsl.map_signal("q4_drive", "q4/drive")
    dsl.map_signal("q5_drive", "q5/drive")

In [ ]:
exp_trig_mark = experiment_trig_mark(5)
compiled_experiment_trig_mark = session.compile(exp_trig_mark)

In [ ]:
plot_simulation(compiled_experiment_trig_mark, start_time=0, length=5e-6)

In [ ]:
_ = session.run(compiled_experiment_trig_mark)

## Sweeping the voltage offset

Here we show you how to sweep the voltage offset on an `rf` channel of the HDAWG. We also play a 200-ns Gaussian pulse on the `SIGOUT/0` output of the HDAWG.

In [ ]:
@dsl.experiment(signals=["q0_flux"])
def experiment_voltage_offset(count):
    with dsl.sweep(
        name="voltage_offset_sweep",
        parameter=SweepParameter("voltages", np.arange(0, 1.05, 0.1)),
    ) as voltage_offset_sweep:
        with dsl.acquire_loop_rt(
            count=count,
            averaging_mode=AveragingMode.CYCLIC,
            acquisition_type=AcquisitionType.INTEGRATION,
        ):
            with dsl.section(name="rf-signal_q0"):
                dsl.play(
                    signal="q0_flux",
                    pulse=pulse_library.gaussian(amplitude=1, length=200e-9),
                )
                dsl.delay("q0_flux", time=0.5e-6)

    # Set the voltage sweep in the experiment calibration
    exp_calibration = dsl.experiment_calibration()
    exp_calibration["q0_flux"] = SignalCalibration(voltage_offset=voltage_offset_sweep)

    # Map the ExperimentSignals "q0_flux" the logical signal line name defined in the `DeviceSetup`
    dsl.map_signal("q0_flux", "q0/flux")

In [ ]:
exp_voltage_offset = experiment_voltage_offset(5)
compiled_experiment_voltage_offset = session.compile(exp_voltage_offset)
_ = session.run(compiled_experiment_voltage_offset)